In [15]:
import pandas

In [56]:
projets = [
    [-14, 90, 0, 40, 100],
    [129, 100, 0, 0, 0],
    [-10, 50, 0, 10, 100],
    [44, 90, 0, 5, 20],
    [-14, 100, 0, 20, 40]
]
p = [50, 24, 1, 24, 20]
q = [25, 16, 0, 12, 10]
importance = [1, 1, 1, 1, 1]
veto = [100, 60, 2, 48, 90]

###########
projects_names = ['P' + str(i + 1) for i in range(len(projets))]
criterias_names = ['C' + str(i + 1) for i in range(len(projets[0]))]

df = pandas.DataFrame(data=projets + [p, q, importance, veto],
                      columns=criterias_names,
                      index=projects_names + ['Préférence', 'Indifférence', 'Importance', 'Veto'])

p=0
q=0

df

In [116]:
k = df.loc["Importance"].sum()

concordance = pandas.DataFrame(index=projects_names,
                               columns=projects_names)

for a in projects_names:
    for b in projects_names:
        s = 0
        for j in criterias_names:
            if df.loc[a][j] >= df.loc[b][j] - df.loc["Indifférence"][j]:
                indice_concordance = 1
            elif df.loc[a][j] <= df.loc[b][j] - df.loc["Préférence"][j]:
                indice_concordance = 0
            else:
                indice_concordance = (df.loc["Préférence"][j] + df.loc[a][j] - df.loc[b][j]) / (df.loc["Préférence"][j] - df.loc["Indifférence"][j])
            s += df.loc["Importance"][j] * indice_concordance
        concordance.loc[a][b] = 1/k * s

concordance

In [117]:
credibilite = pandas.DataFrame(index=projects_names,
                               columns=projects_names)


for a in projects_names:
    for b in projects_names:
        discordances = pandas.Series(index = criterias_names)
        for j in criterias_names:
            if df.loc[a][j] >= df.loc[b][j] - df.loc["Préférence"][j]:
                discordances[j] = 0
            elif df.loc[a][j] <= df.loc[b][j] - df.loc["Veto"][j]:
                discordances[j] = 1
            else:
                discordances[j] = (df.loc[b][j] - df.loc[a][j] - df.loc["Préférence"][j]) / (df.loc["Veto"][j] - df.loc["Préférence"][j])
                
        J = discordances[discordances > concordance.loc[a][b]].index
        
        if len(J) == 0:
            credibilite.loc[a][b] = concordance.loc[a][b]
        else:
            credibilite.loc[a][b] = concordance.loc[a][b] * ((1 - discordances[J]) / (1 - concordance.loc[a][b])).product()

credibilite